<a href="https://colab.research.google.com/github/Amphibian27/Natural-Language-Processing/blob/master/Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
warnings.filterwarnings('ignore', category = DeprecationWarning)
warnings.filterwarnings('ignore', category = UserWarning)

In [113]:
import os
import re 

import multiprocessing
from nltk.tokenize import word_tokenize,WhitespaceTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer,CountVectorizer
from xgboost import XGBClassifier
import string
from string import punctuation
from nltk.corpus import stopwords
import nltk.corpus.util
from sklearn.tree import DecisionTreeClassifier
from nltk.classify import NaiveBayesClassifier



In [ ]:
cores = multiprocessing.cpu_count() - 1

## Data Load

In [126]:
df1 = pd.read_csv(r'Usecase3_Dataset.csv',encoding = 'latin1')


In [127]:
print(df1.columns)
#Length of Dataframe
print(len(df1))
#Rows not containing any text data
print(len(df1[df1["text"].isna() == True]))#0 null values

Index(['airline_sentiment', 'airline', 'text'], dtype='object')
14640
0


## EDA

In [ ]:
df1['airline_sentiment'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Number of Complaints', 
                                                                title='Number complaints in each product')

# Data Cleaning

In [128]:
#Removing handles
df1['cleaned_text'] = df1["text"].apply(lambda row: re.sub(r'@[^\s]+','',row))


In [129]:
#Remove all links
df1['cleaned_text'] = df1["cleaned_text"].apply(lambda row: re.sub(r'http[s]?://\S+', '',str(row)))
df1['cleaned_text'][7]

' Really missed a prime opportunity for Men Without Hats parody, there. '

In [130]:
#Remove special characters excluding apostrophe
df1['cleaned_text'] = df1["cleaned_text"].apply(lambda row: re.sub(r"[^A-Za-z0-9']+", ' ',row))
df1['cleaned_text'][27]

" do you miss me Don't worry we'll be together very soon "

In [131]:
#remove leading and trailing blanks
df1['cleaned_text'] = df1["cleaned_text"].apply(lambda row: row.strip(' '))
df1['cleaned_text'][27]

"do you miss me Don't worry we'll be together very soon"

In [132]:
df1.to_csv('cleaned_data.csv')

In [133]:
df1.shape

(14640, 4)

In [134]:
#Tokenize
df1['cleaned_tokens'] = df1['cleaned_text'].apply(lambda row: row.split(' '))

In [135]:

stopword = nltk.corpus.stopwords.words('english')
#Removing stopwords from tokens
df1["cleaned_tokens"] = df1["cleaned_tokens"].apply(lambda row :[item for item in row if item not in stopword])


In [136]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df1['lemmatized_tokens'] = df1['cleaned_tokens'].apply(lambda x: stemming(x))
df1.head()##Results not satisfactory use wordnet lemmatizer

,airline_sentiment,airline,text,cleaned_text,cleaned_tokens,lemmatized_tokens
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,What said,"[What, said]","[what, said]"
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experienc...,"[plus, added, commercials, experience, tacky]","[plu, ad, commerci, experi, tacki]"
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,I didn't today Must mean I need to take anothe...,"[I, today, Must, mean, I, need, take, another,...","[I, today, must, mean, I, need, take, anoth, t..."
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,it's really aggressive to blast obnoxious ente...,"[really, aggressive, blast, obnoxious, enterta...","[realli, aggress, blast, obnoxi, entertain, gu..."
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,"[really, big, bad, thing]","[realli, big, bad, thing]"


In [141]:
#Join for wordnet lemmatization
df1['cleaned_f']  = df1['cleaned_tokens'].apply(' '.join)


In [142]:
#lemmatization
wnl = WordNetLemmatizer()
df1["cleaned_f"] = df1["cleaned_f"].apply(lambda row: wnl.lemmatize(row))
df1.head()

,airline_sentiment,airline,text,cleaned_text,cleaned_tokens,lemmatized_tokens,cleaned_f
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,What said,"[What, said]","[what, said]",What said
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experienc...,"[plus, added, commercials, experience, tacky]","[plu, ad, commerci, experi, tacki]",plus added commercials experience tacky
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,I didn't today Must mean I need to take anothe...,"[I, today, Must, mean, I, need, take, another,...","[I, today, must, mean, I, need, take, anoth, t...",I today Must mean I need take another trip
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,it's really aggressive to blast obnoxious ente...,"[really, aggressive, blast, obnoxious, enterta...","[realli, aggress, blast, obnoxi, entertain, gu...",really aggressive blast obnoxious entertainmen...
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,"[really, big, bad, thing]","[realli, big, bad, thing]",really big bad thing


In [143]:
#Tokenize again
df1['cleaned_f'] = df1['cleaned_f'].apply(lambda row: row.split(' '))
df1.head()

,airline_sentiment,airline,text,cleaned_text,cleaned_tokens,lemmatized_tokens,cleaned_f
0,neutral,Virgin America,@VirginAmerica What @dhepburn said.,What said,"[What, said]","[what, said]","[What, said]"
1,positive,Virgin America,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experienc...,"[plus, added, commercials, experience, tacky]","[plu, ad, commerci, experi, tacki]","[plus, added, commercials, experience, tacky]"
2,neutral,Virgin America,@VirginAmerica I didn't today... Must mean I n...,I didn't today Must mean I need to take anothe...,"[I, today, Must, mean, I, need, take, another,...","[I, today, must, mean, I, need, take, anoth, t...","[I, today, Must, mean, I, need, take, another,..."
3,negative,Virgin America,@VirginAmerica it's really aggressive to blast...,it's really aggressive to blast obnoxious ente...,"[really, aggressive, blast, obnoxious, enterta...","[realli, aggress, blast, obnoxi, entertain, gu...","[really, aggressive, blast, obnoxious, enterta..."
4,negative,Virgin America,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,"[really, big, bad, thing]","[realli, big, bad, thing]","[really, big, bad, thing]"


In [146]:
#Filtering the empty strings from the list of tokens
df1["cleaned_f"] = df1["cleaned_f"].apply(lambda row: list(filter(None,row)))
df1.shape

(14640, 7)

In [148]:
#Final join with clean text
df1['cleaned_f']  = df1['cleaned_f'].apply(' '.join)

# Sentiment Analysis 


## Dictionary based approach
## Verification of Sentiments (Classification vs Vader)

In [254]:
# create training and testing vars
y = df1.airline_sentiment
X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size=0.3,random_state=42)
print(X_train.shape), print(y_train.shape)
print (X_test.shape), print(y_test.shape)

(10248, 8)
(10248,)
(4392, 8)
(4392,)


(None, None)

In [115]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [150]:
analyzer = SentimentIntensityAnalyzer()

In [192]:
# Assigning Sentiments based on Polarity Score
for item in X_test['text']:   
    i = X_test.index.values[X_test['text'] == item]
    scores = analyzer.polarity_scores(item)
    if (scores['compound'] < 0.05) & (scores['compound'] > -0.05):
        X_test.loc[i,'vader_Senti'] = 'neutral'
    elif scores['compound'] <= -0.05:
        X_test.loc[i,'vader_Senti'] = 'negative'
    elif scores['compound'] >= 0.05:
        X_test.loc[i,'vader_Senti'] = 'positive'

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/loc

In [193]:
#Encoding labels 
le = LabelEncoder()
vader_in = le.fit_transform(X_test['airline_sentiment'])
vader_out = le.transform(X_test['vader_Senti'])

In [194]:
accuracy_score(y_true = vader_out, y_pred  = vader_in)
#Dictionary based approach for benchmarking, received  53% accuracy 

0.5371129326047359

# Machine Learning based approaches

In [183]:
cv = CountVectorizer()
x_train = cv.fit_transform(X_train['cleaned_f'])
x_test = cv.transform(X_test['cleaned_f'])

In [184]:
tfidf = TfidfTransformer()
x_train = tfidf.fit_transform(x_train)
x_test = tfidf.transform(x_test)

In [185]:
#Converting sparse matrix to dense matrix
x_train = x_train.todense()
x_test = x_test.todense()

### Decision Tree Classifier

In [186]:
#DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [189]:
X_test['Decision Tree'] = clf.predict(x_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [190]:
#Encoding labels 
le = LabelEncoder()
actual = le.fit_transform(X_test['airline_sentiment'])
predicted = le.transform(X_test['Decision Tree'])

In [191]:
accuracy_score(y_true = actual, y_pred  = predicted)
#Decision tree managed to improve accuracy from 54% to 70%

0.7019581056466302

### XGBoost Classifier

In [195]:
#Classification using XGBoost Classifier
xgb = XGBClassifier(random_state = 21,n_jobs = cores)
xgb.fit(x_train,y_train)
X_test['XGB'] = xgb.predict(x_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [201]:
#Encoding labels 
le = LabelEncoder()
actual = le.fit_transform(X_test['airline_sentiment'])
predicted = le.transform(X_test['XGB'])

In [202]:
accuracy_score(y_true = actual, y_pred  = predicted)
#Decision tree managed to improve accuracy from 54% to 70%

0.7015027322404371

### Support Vector Classifier

In [206]:
#Classification Using Support Vector Classifier
clfSVM = LinearSVC()
clfSVM.fit(x_train,y_train)
X_test['SVM'] = clfSVM.predict(x_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [207]:
#Encoding labels 
le = LabelEncoder()
actual = le.fit_transform(X_test['airline_sentiment'])
predicted = le.transform(X_test['SVM'])

In [208]:
accuracy_score(y_true = actual, y_pred  = predicted)
#Decision tree managed to improve accuracy from 54% to 70%

0.5412112932604736

### Deeplearning - Keras

In [300]:
# Import modules
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence  import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import SpatialDropout1D 
from keras.callbacks import EarlyStopping 
from keras.layers import Dense,Dropout
from keras.models import Sequential


In [311]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 10000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 128
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df1['cleaned_f'].values)
word_index = tokenizer.word_index
print('%s unique tokens.' % len(word_index))

13312 unique tokens.


In [312]:
X = tokenizer.texts_to_sequences(df1['cleaned_f'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (14640, 250)


In [313]:
Y = pd.get_dummies(df1['airline_sentiment']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (14640, 3)


In [314]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(10248, 250) (10248, 3)
(4392, 250) (4392, 3)


In [317]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 50
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Train on 9223 samples, validate on 1025 samples
Epoch 1/50
9223/9223 [==============================] - 74s 8ms/step - loss: 0.7828 - accuracy: 0.6685 - val_loss: 0.6372 - val_accuracy: 0.7444
Epoch 2/50
9223/9223 [==============================] - 75s 8ms/step - loss: 0.4979 - accuracy: 0.8100 - val_loss: 0.5432 - val_accuracy: 0.7737
Epoch 3/50
9223/9223 [==============================] - 75s 8ms/step - loss: 0.3518 - accuracy: 0.8695 - val_loss: 0.5535 - val_accuracy: 0.7717
Epoch 4/50
9223/9223 [==============================] - 75s 8ms/step - loss: 0.2707 - accuracy: 0.9037 - val_loss: 0.6225 - val_accuracy: 0.7717
Epoch 5/50
9223/9223 [==============================] - 75s 8ms/step - loss: 0.2093 - accuracy: 0.9251 - val_loss: 0.6715 - val_accuracy: 0.7610


In [318]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

4392/4392 [==============================] - 8s 2ms/step
Test set
  Loss: 0.681
  Accuracy: 0.779
